In [1]:
%load_ext nb_black
# !pip install nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np
import warnings
from matplotlib import pyplot as plt
from sklearn import metrics as m
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE  # doctest: +NORMALIZE_WHITESPACE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

# from sklearn.svm import SVC

from thundersvm import SVC

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
import xgboost as xgb
from sklearn.pipeline import make_pipeline
from mlxtend.feature_selection import ColumnSelector
from sklearn import model_selection
from mlxtend.classifier import StackingClassifier


warnings.filterwarnings("ignore")
randomseed = 7

Using TensorFlow backend.


<IPython.core.display.Javascript object>

In [3]:
x_original = pd.read_csv("../../XLable_onlyDiabeticRemoved.txt")

conditions = [
    (x_original.L100800 < 100),
    (x_original.L100800 >= 100) & (x_original.L100800 < 126),
    (x_original.L100800 >= 126),
]
choices = [0, 1, 2]
x_original["CLASS"] = np.select(conditions, choices, default=0)
x_original = x_original[
    [
        "Unnamed: 0",
        "L100800",
        "L104600",
        "L103000",
        "S000300",
        "L101700",
        "L100700",
        "FIELD_33",
        "FIELD_38",
        "FIELD_40",
        "FIELD_31",
        "SEX",
        "AGE",  #'CLASS',
        "FIELD_16",
        "FIELD_23",
        "FIELD_15",
        "FIELD_22",
        "FIELD_17",
        "FIELD_24",
    ]
]

<IPython.core.display.Javascript object>

In [4]:
y_original = pd.read_csv("../../TargetLable_onlyDiabeticRemoved.txt")

conditions = [
    (y_original.L100800 < 100),
    (y_original.L100800 >= 100) & (y_original.L100800 < 126),
    (y_original.L100800 >= 126),
]

choices = [0, 1, 2]
y_original["CLASS"] = np.select(conditions, choices, default=0)

y_original = y_original[["Unnamed: 0", "CLASS"]]

<IPython.core.display.Javascript object>

In [5]:
data = pd.merge(
    x_original, y_original, how="inner", left_on="Unnamed: 0", right_on="Unnamed: 0"
)

<IPython.core.display.Javascript object>

In [6]:
# filter the data set
data = data[data.FIELD_16 != 1]  # exclude people who are diagnosed for (diabetes)
data = data[data.FIELD_23 != 1]  # exclude people who are on medication for diabetes
data.head()

,Unnamed: 0,L100800,L104600,L103000,S000300,L101700,L100700,FIELD_33,FIELD_38,FIELD_40,FIELD_31,SEX,AGE,FIELD_16,FIELD_23,FIELD_15,FIELD_22,FIELD_17,FIELD_24,CLASS
0,0,77.0,NaN,53.0,20.1,13.0,3.0,1.0,1.0,0.0,1.0,1.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1,89.0,NaN,53.0,19.7,14.0,3.0,1.0,1.0,1.0,0.0,1.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,2,78.0,5.28,41.0,20.2,15.0,3.8,1.0,2.0,1.0,0.0,1.0,46.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,3,98.0,NaN,80.0,24.7,13.0,4.1,1.0,0.0,0.0,0.0,1.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,4,95.0,NaN,58.0,24.8,10.0,3.7,1.0,0.0,0.0,1.0,1.0,51.0,0.0,0.0,0.0,0.0,0.0,0.0,0


<IPython.core.display.Javascript object>

In [7]:
data = data.dropna()
print(data.shape)
data = data[
    [
        "L100800",
        "L104600",
        "L103000",
        "S000300",
        "L101700",
        "L100700",
        "FIELD_33",
        "FIELD_38",
        "FIELD_40",
        "FIELD_31",
        "SEX",
        "AGE",
        "CLASS",
    ]
]
data.head()

(62871, 20)


,L100800,L104600,L103000,S000300,L101700,L100700,FIELD_33,FIELD_38,FIELD_40,FIELD_31,SEX,AGE,CLASS
2,78.0,5.28,41.0,20.2,15.0,3.8,1.0,2.0,1.0,0.0,1.0,46.0,0
5,90.0,5.74,50.0,25.5,12.0,3.4,1.0,0.0,1.0,0.0,1.0,52.0,0
10,86.0,5.83,45.0,21.2,17.0,3.9,1.0,0.0,1.0,1.0,1.0,37.0,0
11,86.0,4.73,54.0,22.0,30.0,4.2,1.0,2.0,3.0,0.0,1.0,39.0,0
15,106.0,5.97,71.0,23.5,13.0,5.4,1.0,0.0,4.0,0.0,1.0,69.0,0


<IPython.core.display.Javascript object>

In [8]:
data.groupby(by="CLASS").size()

CLASS
0    41017
1    20487
2     1367
dtype: int64

<IPython.core.display.Javascript object>

In [9]:
# add age group to the dataset
data["ageG"] = (data.AGE / 10).astype(int)

# rearange position
temp = data.CLASS
data = data.drop(columns=["CLASS"])
data["CLASS"] = temp

data.head()

,L100800,L104600,L103000,S000300,L101700,L100700,FIELD_33,FIELD_38,FIELD_40,FIELD_31,SEX,AGE,ageG,CLASS
2,78.0,5.28,41.0,20.2,15.0,3.8,1.0,2.0,1.0,0.0,1.0,46.0,4,0
5,90.0,5.74,50.0,25.5,12.0,3.4,1.0,0.0,1.0,0.0,1.0,52.0,5,0
10,86.0,5.83,45.0,21.2,17.0,3.9,1.0,0.0,1.0,1.0,1.0,37.0,3,0
11,86.0,4.73,54.0,22.0,30.0,4.2,1.0,2.0,3.0,0.0,1.0,39.0,3,0
15,106.0,5.97,71.0,23.5,13.0,5.4,1.0,0.0,4.0,0.0,1.0,69.0,6,0


<IPython.core.display.Javascript object>

In [10]:
# from sklearn.cluster import KMeans
# import numpy as np

# kmeans = KMeans(n_clusters=3, random_state=0).fit(data)
# kmeans.labels_

# kmeans.cluster_centers_

<IPython.core.display.Javascript object>

In [11]:
# data["cluster"] = kmeans.predict(data)
# data.head()

<IPython.core.display.Javascript object>

In [12]:
# # rearange position
# temp = data.CLASS
# data = data.drop(columns=["CLASS"])
# data["CLASS"] = temp
# data.head()

<IPython.core.display.Javascript object>

In [13]:
diabetic = data[data.CLASS == 2]
prediabetic = data[data.CLASS == 1]
normal = data[data.CLASS == 0]

print(diabetic.shape[0], prediabetic.shape[0], normal.shape[0])

1367 20487 41017


<IPython.core.display.Javascript object>

In [14]:
diabetic_test = diabetic.sample(200, random_state=randomseed)
prediabetic_test = prediabetic.sample(200, random_state=randomseed)
normal_test = normal.sample(200, random_state=randomseed)
test = pd.concat([diabetic_test, prediabetic_test, normal_test])

diabetic_train = diabetic.drop(diabetic_test.index)
prediabetic_train = prediabetic.drop(prediabetic_test.index)
# .sample(
#     2 * diabetic_train.shape[0], random_state=randomseed
# )
normal_train = normal.drop(normal_test.index).sample(
    prediabetic_train.shape[0],
    random_state=randomseed
    #     10 * diabetic_train.shape[0], random_state=randomseed
)
train = pd.concat([diabetic_train, diabetic_train, prediabetic_train, normal_train])

<IPython.core.display.Javascript object>

In [15]:
xtrain = train.iloc[:, :-1]
ytrain = train.iloc[:, -1]
xtest = test.iloc[:, :-1]
ytest = test.iloc[:, -1]

<IPython.core.display.Javascript object>

In [16]:
# xtrain = xtrain.drop(columns=["FIELD_38"])
# xtest = xtest.drop(columns=["FIELD_38"])

<IPython.core.display.Javascript object>

In [17]:
from sklearn.preprocessing import MinMaxScaler 
scaler = MinMaxScaler()
xtrain=scaler.fit_transform(xtrain)
xtest=scaler.transform(xtest)


<IPython.core.display.Javascript object>

In [18]:
from imblearn.over_sampling import SMOTE,SMOTENC  # doctest: +NORMALIZE_WHITESPACE
randomseed=42

sm = SMOTENC(random_state=randomseed,categorical_features=[4,5],sampling_strategy='minority')
X_res, y_res = sm.fit_resample(xtrain, ytrain)

print("Resampled dataset shape %s" % Counter(y_res))
print(
    y_res[y_res == 0].shape[0], y_res[y_res == 1].shape[0], y_res[y_res == 2].shape[0]
)
print(X_res.shape, y_res.shape)

xtrain = X_res
ytrain = y_res


Resampled dataset shape Counter({2: 20287, 1: 20287, 0: 20287})
20287 20287 20287
(60861, 13) (60861,)


<IPython.core.display.Javascript object>

In [19]:
train.groupby(by="CLASS").size()

CLASS
0    20287
1    20287
2     2334
dtype: int64

<IPython.core.display.Javascript object>

# Clustering approach

In [297]:
rf_12 = RandomForestClassifier(
    random_state=randomseed,
    n_estimators=100,
    max_depth=12,
    min_samples_split=2,
    min_samples_leaf=10,
    max_features="auto",
)
xgb_12 = xgb.XGBClassifier(objective="binary:logistic", random_state=randomseed)


<IPython.core.display.Javascript object>

In [298]:

SVC_12 = SVC(
    C=70,
#     cache_size=200,
#     class_weight=None,
#     coef0=0.0,
#     decision_function_shape="ovr",
#     degree=3,
#     gamma="auto_deprecated",
#     kernel="linear",
#     max_iter=-1,
    probability=True,
#     random_state=randomseed,
#     shrinking=True,
#     tol=0.001,
#     verbose=False
)

# SVC(gamma='auto')


<IPython.core.display.Javascript object>

In [299]:
# SVC_12 = SVC(gamma="auto")

<IPython.core.display.Javascript object>

In [300]:
SVC(
    C=70,
    cache_size=200,
    class_weight=None,
    coef0=0.0,
    decision_function_shape="ovr",
    degree=3,
    gamma="auto_deprecated",
    gpu_id=0,
    kernel="linear",
    max_iter=-1,
    max_mem_size=-1,
    n_jobs=-1,
    probability=1,
    random_state=42,
    shrinking=True,
    tol=0.001,
    verbose=False,
)

SVC(C=70, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated', gpu_id=0,
    kernel='linear', max_iter=-1, max_mem_size=-1, n_jobs=-1, probability=1,
    random_state=42, shrinking=True, tol=0.001, verbose=False)

<IPython.core.display.Javascript object>

In [301]:
ytest.shape

(600,)

<IPython.core.display.Javascript object>

In [302]:
clf = SVC()
clf.fit(xtrain, ytrain)

SVC(C=1.0, cache_size=None, class_weight={}, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='auto', gpu_id=0,
    kernel='rbf', max_iter=-1, max_mem_size=-1, n_jobs=-1, probability=False,
    random_state=None, shrinking=False, tol=0.001, verbose=False)

<IPython.core.display.Javascript object>

In [303]:
col12 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

<IPython.core.display.Javascript object>

In [304]:
col5 = [0, 1, 3, 10, 11, 12]

<IPython.core.display.Javascript object>

In [309]:
# generate the weak models
estimators2 = []


# estimators2.append((("rf_base5" ),make_pipeline(ColumnSelector(cols=col5), rf_12)))
# estimators2.append(("xgb_base5",make_pipeline(ColumnSelector(cols=col5), xgb_12)))
# estimators2.append(("SVC_base5" ,make_pipeline(ColumnSelector(cols=col5), SVC_12)))


estimators2.append((("rf_base" ),rf_12))
estimators2.append(("xgb_base", xgb_12))
estimators2.append(("SVC_base" ,SVC_12))


<IPython.core.display.Javascript object>

In [310]:
len(estimators2)

3

<IPython.core.display.Javascript object>

In [319]:
from mlxtend.classifier import StackingClassifier

lr = LogisticRegression()
sclf = StackingClassifier(
    classifiers=[pipe2, rf_12, xgb_12, SVC_12, pipe1], meta_classifier=lr, verbose=2
)

<IPython.core.display.Javascript object>

In [320]:
sclf.fit(xtrain, ytrain)

Fitting 5 classifiers...
Fitting classifier1: pipeline (1/5)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
                                drop_axis=False)),
                ('svc',
                 SVC(C=1.0, cache_size=None, class_weight=None, coef0=0.0,
                     decision_function_shape='ovo', degree=3, gamma='auto',
                     gpu_id=0, kernel='rbf', max_iter=-1, max_mem_size=-1,
                     n_jobs=-1, probability=False, random_state=None,
                     shrinking=False, tol=0.001, verbose=False))],
         verbose=False)
Fitting classifier2: randomforestclassifier (2/5)
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=12, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_spl

StackingClassifier(average_probas=False,
                   classifiers=[Pipeline(memory=None,
                                         steps=[('columnselector',
                                                 ColumnSelector(cols=[0, 1, 2,
                                                                      3, 4, 5,
                                                                      6, 7, 8,
                                                                      9, 10, 11,
                                                                      12],
                                                                drop_axis=False)),
                                                ('svc',
                                                 SVC(C=1.0, cache_size=None,
                                                     class_weight=None,
                                                     coef0=0.0,
                                                     decision_function_shape='ovo',
         

<IPython.core.display.Javascript object>

In [321]:
y_pred = sclf.predict(xtest)
print("accuracy \t", m.accuracy_score(ytest, y_pred))

confmatrx = pd.DataFrame(m.confusion_matrix(ytest, y_pred))
confmatrx.head()

accuracy 	 0.7366666666666667


,0,1,2
0,148,52,0
1,49,135,16
2,2,39,159


<IPython.core.display.Javascript object>

In [280]:
from mlxtend.classifier import EnsembleVoteClassifier

<IPython.core.display.Javascript object>

In [316]:
clf_1 = SVC()

clf_2 = SVC()

pipe1 = make_pipeline(ColumnSelector(cols=col5), clf_1)

pipe2 = make_pipeline(ColumnSelector(cols=col12), clf_2)

eclf = EnsembleVoteClassifier(
    clfs=[pipe1, pipe2], weights=[1, 1], refit=False, voting="hard", verbose=2
)

labels = ["svm", "xgboost", "ensemble"]

eclf.fit(xtrain, ytrain)

EnsembleVoteClassifier(clfs=[Pipeline(memory=None,
                                      steps=[('columnselector',
                                              ColumnSelector(cols=[0, 1, 3, 10,
                                                                   11, 12],
                                                             drop_axis=False)),
                                             ('svc',
                                              SVC(C=1.0, cache_size=None,
                                                  class_weight=None, coef0=0.0,
                                                  decision_function_shape='ovo',
                                                  degree=3, gamma='auto',
                                                  gpu_id=0, kernel='rbf',
                                                  max_iter=-1, max_mem_size=-1,
                                                  n_jobs=-1, probability=False,
                                                  ran

<IPython.core.display.Javascript object>

In [284]:
eclf.predict(xtest)

OSError: exception: access violation reading 0x0000000000000000

<IPython.core.display.Javascript object>

In [324]:
eclf2 = VotingClassifier(
    estimators=estimators2,
    #     weights=[1, 1, 1, 1, 1, 1],
    voting="soft"
    # , n_jobs=-1
)
eclf2.fit(xtrain, ytrain)

VotingClassifier(estimators=[('rf_base',
                              RandomForestClassifier(bootstrap=True,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=12,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=10,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                                     n_estimators=100,
                                                     n_jobs=None,
      

<IPython.core.display.Javascript object>

In [326]:
y_pred = eclf.predict(xtest)
print("accuracy \t", m.accuracy_score(ytest, y_pred))

confmatrx = pd.DataFrame(m.confusion_matrix(ytest, y_pred))
confmatrx.head()

OSError: exception: access violation reading 0x0000000000000000

<IPython.core.display.Javascript object>

In [ ]:
print(classification_report(ytest, y_pred))

In [277]:
model = estimators2[0][1]
# model = SVC_12

<IPython.core.display.Javascript object>

In [236]:
# pd.DataFrame.sparse.from_spmatrix(xtrain)

<IPython.core.display.Javascript object>

In [237]:
# import scipy.sparse as sparse

# sparse.csr_matrix(xtrain)

<IPython.core.display.Javascript object>

In [238]:
# sparse.csr_matrix(list(ytrain)).reshape((-1, 1)).shape

<IPython.core.display.Javascript object>

In [239]:
# from sklearn.preprocessing import LabelEncoder

# lb = LabelEncoder()
# ytrain = lb.fit_transform(ytrain)

<IPython.core.display.Javascript object>

In [240]:
# xtrain = xtrain.astype(float)
# ytrain = ytrain.astype(int)

<IPython.core.display.Javascript object>

In [241]:
# np.array(xtrain[:, 10])

<IPython.core.display.Javascript object>

In [242]:
# for i in range(12):
#     print(type(xtrain[:, i][0]))
#     print((xtrain[:, i].any()))

<IPython.core.display.Javascript object>

In [243]:
# model.fit(
#     sparse.csr_matrix(xtrain),
#     sparse.csr_matrix(list(np.array(ytrain))).reshape((-1, 1)),
# )
# model

<IPython.core.display.Javascript object>

In [256]:
model = estimators2[2][1]
model.fit(xtrain, ytrain)
model

Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 3, 10, 11, 12], drop_axis=False)),
                ('svc',
                 SVC(C=70, cache_size=None, class_weight={}, coef0=0.0,
                     decision_function_shape='ovo', degree=3, gamma='auto',
                     gpu_id=0, kernel='rbf', max_iter=-1, max_mem_size=-1,
                     n_jobs=-1, probability=False, random_state=None,
                     shrinking=False, tol=0.001, verbose=False))],
         verbose=False)

<IPython.core.display.Javascript object>

In [257]:
y_pred = model.predict(xtest)
print("accuracy \t", m.accuracy_score(ytest, y_pred))

confmatrx = pd.DataFrame(m.confusion_matrix(ytest, y_pred))
confmatrx.head()

accuracy 	 0.7666666666666667


,0,1,2
0,158,42,0
1,48,128,24
2,2,24,174


<IPython.core.display.Javascript object>

In [258]:
# type(model.predict_proba(xtest)[0][0])

<IPython.core.display.Javascript object>

In [259]:
print(classification_report(ytest, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.79      0.77       200
           1       0.66      0.64      0.65       200
           2       0.88      0.87      0.87       200

    accuracy                           0.77       600
   macro avg       0.77      0.77      0.77       600
weighted avg       0.77      0.77      0.77       600



<IPython.core.display.Javascript object>

In [218]:
xtrain.shape

(60861, 13)

<IPython.core.display.Javascript object>